In [37]:
from dotenv import load_dotenv

load_dotenv()

True

### <b>문제 7-1 : LangGraph ReAct Agent 실습 연습문제 (Vector DB + Tool 연동) </b>
<b>문제 설명</b><br>
: prebuilt 컴포넌트를 사용하지 않고, StateGraph와 조건부 엣지를 활용하여 사용자 정의 ReAct Agent를 구현하세요. Agent는 카페 메뉴 검색 도구를 활용하여 고객의 음료 및 디저트 관련 질문에 답변하며, LLM 응답에 도구 호출이 포함되어 있으면 도구를 실행하고, 그렇지 않으면 답변을 완료하는 구조로 작동해야 합니다.

<b>핵심 개념</b>
1. 카페 도메인 특화 Agent
2. MessagesState 기반 상태 관리
3. 조건부 엣지 함수

<b>요구사항</b>
- 카페 상태 정의: MessagesState를 상속한 Agent 상태 정의
- 카페 Agent 노드: 카페 메뉴 관련 LLM 응답 생성 노드
- 카페 메뉴 Tool 노드: search_cafe_menu를 사용한 도구 실행 노드
- 조건부 분기: tools_condition을 사용한 자동 분기
- 순환 구조: 도구 실행 후 다시 Agent로 돌아가는 구조


In [ ]:
from langgraph.graph import StateGraph, END
from langgraph.graph import MessagesState
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langgraph.prebuilt import tools_condition

# AgentState 정의
class AgentState(MessagesState):
    pass

# LLM 준비
llm = ChatOpenAI(model="gpt-4o", temperature=0)

# 프롬프트
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 카페 메뉴 전문가입니다. 질문에 성실히 답변하세요. 필요 시 도구를 사용해 메뉴를 검색할 수 있습니다."),
    ("placeholder", "{messages}")
])

agent_chain = prompt | llm

# 벡터 DB 로드
embeddings_model = OllamaEmbeddings(model="bge-m3:latest")
menu_db = FAISS.load_local(
    "./db/cafe_db",
    embeddings_model,
    allow_dangerous_deserialization=True
)

# Tool 정의 (벡터 DB 검색 기반)
@tool
def search_cafe_menu(item: str) -> str:
    """카페 메뉴를 벡터 DB에서 검색하여 정보를 반환합니다."""
    docs = menu_db.similarity_search(item, k=3)
    if not docs:
        return f"{item} 메뉴 정보를 찾을 수 없습니다."
    
    result = "\n".join([f"- {doc.page_content}" for doc in docs])
    return f"'{item}' 메뉴 검색 결과:\n{result}"

tools = [search_cafe_menu]

# Tool Node
def tool_node(state: AgentState) -> AgentState:
    last_tool_call = state.get_last_tool_call()
    if last_tool_call:
        tool_name = last_tool_call["name"]
        args = last_tool_call["args"]
        for t in tools:
            if t.name == tool_name:
                result = t.invoke(args)
                state.append(ToolMessage(content=result, tool_call_id=last_tool_call["id"]))
    return state

# Agent Node
def agent_node(state):
    response = agent_chain.invoke(state)
    state["messages"].append(response) 
    return state


# StateGraph 구성
builder = StateGraph(AgentState)

# 노드 등록
builder.add_node("agent", agent_node)
builder.add_node("tool_node", tool_node)

# 엣지 구성
builder.set_entry_point("agent")
builder.add_conditional_edges(
    "agent",
    tools_condition,
    path_map={
        "tools": "tool_node",  
        "__end__" : END           
    }
)
builder.add_edge("tool_node", "agent")  

# Graph 컴파일
graph = builder.compile()

test_inputs = [
    "아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.",
    "라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?",
    "디저트 메뉴 중에서 티라미수에 대해 자세히 설명해주세요."
]

for idx, question in enumerate(test_inputs, start=1):
    print(f"\n--- 예시 질문 {idx} ---")
    init_state = AgentState(messages=[
        HumanMessage(content=question)
    ])
    final_state = graph.invoke(init_state)

    for msg in final_state["messages"]:
        print(f"{msg.type}: {msg.content}")



--- 예시 질문 1 ---
human: 아메리카노와 아이스 아메리카노의 차이점과 가격을 알려주세요.
ai: 아메리카노와 아이스 아메리카노는 기본적으로 같은 음료이지만, 온도와 제공 방식에서 차이가 있습니다.

1. **아메리카노**: 에스프레소에 뜨거운 물을 추가하여 만든 음료입니다. 따뜻하게 제공되며, 에스프레소의 진한 맛을 유지하면서도 물의 양에 따라 농도를 조절할 수 있습니다.

2. **아이스 아메리카노**: 에스프레소에 차가운 물과 얼음을 추가하여 만든 음료입니다. 시원하게 즐길 수 있으며, 특히 더운 날씨에 인기가 많습니다.

가격은 카페마다 다를 수 있지만, 일반적으로 아이스 아메리카노가 약간 더 비쌀 수 있습니다. 이는 얼음과 차가운 물을 추가하는 과정에서의 추가 비용 때문일 수 있습니다. 보통 아메리카노는 3,000원에서 5,000원 사이, 아이스 아메리카노는 3,500원에서 5,500원 사이로 판매됩니다. 정확한 가격은 방문하는 카페의 메뉴를 확인하는 것이 좋습니다.

--- 예시 질문 2 ---
human: 라떼 종류에는 어떤 메뉴들이 있고 각각의 특징은 무엇인가요?
ai: 라떼는 에스프레소와 스팀 밀크를 기본으로 하는 커피 음료로, 다양한 변형이 있습니다. 각 라떼 종류는 추가되는 재료나 조리법에 따라 독특한 맛과 특징을 가집니다. 다음은 몇 가지 인기 있는 라떼 종류와 그 특징입니다:

1. **카페 라떼 (Cafe Latte)**:
   - 기본적인 라떼로, 에스프레소에 스팀 밀크를 많이 넣고 얇은 층의 우유 거품을 얹습니다. 부드럽고 크리미한 맛이 특징입니다.

2. **바닐라 라떼 (Vanilla Latte)**:
   - 카페 라떼에 바닐라 시럽을 추가하여 달콤하고 향긋한 바닐라 향이 나는 라떼입니다.

3. **카라멜 라떼 (Caramel Latte)**:
   - 카페 라떼에 카라멜 시럽을 첨가하여 달콤하고 풍부한 카라멜 맛을 즐길 수 있습니다.

4. **헤이즐넛 라떼 (Hazelnut Latte)**:
   - 에스프레